<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Analyzing Mexican Restaurants in Lawrence Park Toronto</font></h1>


## Introduction

For the Final Capstone of the IBM Data Science Course, there is a group of investors taht want to open a new Mexican Restaurant in Toronto, Canada. I have made a previous analysis about the Neighborhoods in Toronto, and the stakeholders decided to review 3 Neighborhoods:Lawrence Park, North Toronto and Rosadale. 
This Notebook will focus on reviewing **Lawrence Park Neiborhood**. I will use other Notebooks to analyze the other 2 Neighborhoods.


## Table of Contents

1.  <a href="#item1">Using Foursquare to find Mexican Restaurants Nearby</a>
2.  <a href="#item2">Explore a Given Mexican Restaurant Venue</a>  
3.  <a href="#item3">Explore Venyes in the loacation</a>  
4.  <a href="#item4">Get Trending Venues</a>  



### Import necessary Libraries


In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version


##### Make sure that you have created a Foursquare developer account and have your credentials handy


In [2]:
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
ACCESS_TOKEN = 'your Foursquare Access Token' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 100
print('Your credentails are saved')


Your credentails are saved


#### Let's Analyze Lawrence Park Neighborhood


We will use the coordinates from our previous lab for this Neighborhood.


### Lawrence Park

In [3]:
latitude = 43.7280205
longitude = -79.3887901
import json

<a id="item1"></a>


## 1. Search for a specific venue category

> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**


#### Now, let's assume that it is lunch time, and you are craving Mexican food. So, let's define a query to search for Mexican food that is within 1000 meters from the Location.


In [4]:
search_query = 'Mexican'
radius = 1000
print(search_query + ' .... OK!')

Mexican .... OK!


#### Define the corresponding URL


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)


#### Send the GET Request and examine the results


In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600469b7c754223a4e10a3e6'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 11}}],
 'response': {'venues': []}}

In [7]:
if len(results['response']['venues']) == 0:
    print('There are not Mexican Restaurants')
    MexicanRestaurants = False
else:
    print('There are {} Mexican Restaurants around Lawrence Park.'.format(len(results['response']['venues'])))
    MexicanRestaurants = True

There are not Mexican Restaurants


## Save Request to a JSON File if there are Mexican Restaurants if not jump to 

In [ ]:
import json
with open(r'MexicanRestaurantsLawrencePark.json', 'w') as f:
    json.dump(results, f)

#### Get relevant part of JSON and transform it into a _pandas_ dataframe


In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

#### Define information of interest and filter dataframe


In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]



In [4]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [ ]:
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

## Saving Mexican Restaurants to CSV File

In [ ]:
dataframe_filtered.to_csv('MexicanLawrencePark.csv', index=False) 

#### Let's visualize the restaurants that are nearby


In [ ]:
dataframe_filtered.name

In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>


## 2. Explore a Given Venue

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**


### A. Let's explore the closest Mexican restaurant -- _Change the name of this Restaurant and Venue ID_


In [ ]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)


#### Send GET request for result


In [ ]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

## Saving JSON Data to CSV file

In [ ]:
with open('AMexicanLawrencePark.json', 'w') as f:
    json.dump(result, f)

### B. Get the venue's overall rating


In [ ]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

That is not a very good rating. Let's check the rating of the second closest Italian restaurant.


In [ ]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

Since this restaurant has no ratings, let's check the third restaurant.


In [ ]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

Since this restaurant has a slightly better rating, let's explore it further.


### C. Get the number of tips


In [ ]:
result['response']['venue']['tips']['count']

### D. Get the venue's tips

> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**


#### Create URL and send GET request. Make sure to set limit to get all tips


In [ ]:
## Ecco Tips
limit = 30 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)

results = requests.get(url).json()
results

#### Get tips and list of associated features


In [ ]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

#### Format column width and display all tips


In [ ]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.


<a id="item3"></a>


## 3. Explore a location

> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**


#### So, we just finished reviewing Mexican Restaurants, let's explore what Venues are 1000 meters away.


In [6]:
latitude = 43.7280205
longitude = -79.3887901

#### Define URL


In [9]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)


#### Send GET request and examine results


In [10]:
import requests

In [11]:
results = requests.get(url).json()
'There are {} around Lawrence Park.'.format(len(results['response']['groups'][0]['items']))

'There are 8 around Lawrence Park.'

#### Get relevant part of JSON


In [12]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '50e6da19e4b0d8a78a0e9794',
  'name': 'Lawrence Park Ravine',
  'location': {'address': '3055 Yonge Street',
   'crossStreet': 'Lawrence Avenue East',
   'lat': 43.72696303913755,
   'lng': -79.39438246708775,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.72696303913755,
     'lng': -79.39438246708775}],
   'distance': 465,
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['3055 Yonge Street (Lawrence Avenue East)',
    'Toronto ON',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d163941735',
    'name': 'Park',
    'pluralName': 'Parks',
    'shortName': 'Park',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/park_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-50

#### Process JSON and convert it to a clean dataframe


In [15]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,id
0,Lawrence Park Ravine,Park,3055 Yonge Street,Lawrence Avenue East,43.726963,-79.394382,"[{'label': 'display', 'lat': 43.72696303913755...",465,CA,Toronto,ON,Canada,"[3055 Yonge Street (Lawrence Avenue East), Tor...",NaN,50e6da19e4b0d8a78a0e9794
1,Granite Club,Gym / Fitness Center,2350 Bayview Ave.,Lawrence Ave. West,43.733043,-79.381986,"[{'label': 'display', 'lat': 43.7330429446901,...",782,CA,Toronto,ON,Canada,"[2350 Bayview Ave. (Lawrence Ave. West), Toron...",M2L 1E4,4b22e0a0f964a520544f24e3
2,Tim Hortons,Coffee Shop,2275 Bayview Ave,NaN,43.727324,-79.379563,"[{'label': 'display', 'lat': 43.72732441416303...",746,CA,Toronto,ON,Canada,"[2275 Bayview Ave, Toronto ON M4N 3M6, Canada]",M4N 3M6,50a14c44e4b0bbf777f5152c
3,Glendon Bookstore,Bookstore,2275 Bayview Ave,Bayview and Lawrence,43.727024,-79.378976,"[{'label': 'display', 'lat': 43.72702356473666...",797,CA,Toronto,ON,Canada,"[2275 Bayview Ave (Bayview and Lawrence), Toro...",NaN,4ca34bc15720b1f77f5f30ef
4,Glendon Forest,Trail,2275 Bayview Ave,Bayview Avenue and Lawrence,43.727226,-79.378413,"[{'label': 'display', 'lat': 43.72722590613304...",839,CA,Toronto,ON,Canada,[2275 Bayview Ave (Bayview Avenue and Lawrence...,NaN,4c824217dc018cfae226cd6c
5,Glendon Rose Garden,College Quad,2275 Bayview Avenue,Lawrence Avenue,43.727334,-79.378222,"[{'label': 'display', 'lat': 43.72733415672508...",853,CA,Toronto,ON,Canada,"[2275 Bayview Avenue (Lawrence Avenue), Toront...",M4N 3M6,4deea1463151ea1d4578cb66
6,Lunik Co-op,Café,2275 Bayview Ave,NaN,43.727311,-79.377835,"[{'label': 'display', 'lat': 43.72731127063538...",884,CA,Toronto,ON,Canada,"[2275 Bayview Ave, Toronto ON, Canada]",NaN,4f29422fe4b09023de491f10
7,Glendon Athletic Club,College Gym,2275 Bayview Ave.,at Lawrence Ave. E,43.728799,-79.376819,"[{'label': 'display', 'lat': 43.7287988983121,...",966,CA,Toronto,ON,Canada,"[2275 Bayview Ave. (at Lawrence Ave. E), Toron...",M4N 3M6,4d49e1e5bdf988bf82fba557


## Let's save the data frame to a CSV file

In [16]:
dataframe_filtered.to_csv('VenuesLawrencePark.csv', index=False) 

#### Let's visualize these items on the map around our location


In [17]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Lawrence Park',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<a ><img src = "https://github.com/kmiralles/Captstone-Data-Science/blob/main/LawrenceParkVenues.png?raw=true" > </a>


<a id="item4"></a>


## 4. Explore Trending Venues

> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**


#### Now, instead of simply exploring the area around this location, we are interested in knowing the venues that are trending at this time, meaning the places with the highest foot traffic. So let's do that and get the trending venues around this location.


## Run the following lines at Breakfast time

In [18]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60046aa20bf2b4487810ec5a'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [19]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [20]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

## Save Trending Venues at Breakfast time to CSV

In [ ]:
trending_venues_df.df.to_csv('TrendingBreakfastLawrencePark.csv', index=False) 

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Lawrence Park',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

## Run the following lines at Lunch time

In [5]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600cc85302db8f06a13be923'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [6]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [7]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

## Save Trending Venues at Lunch time to CSV

In [ ]:
trending_venues_df.df.to_csv('TrendingLunchLawrencePark.csv', index=False) 

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Lawrence Park',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

## Run the following lines at HappyHour time

In [7]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600a1241804dc92cb8fc0e67'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [8]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [9]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

## Save Trending Venues at Happy Hour time to CSV

In [ ]:
trending_venues_df.df.to_csv('TrendingHappyHourLawrencePark.csv', index=False) 

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Lawrence Park',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

## Run the following lines at Dinner time

In [6]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '600a525e1bf90431bac85033'},
 'response': {'venues': []}}

### Check if any venues are trending at this time


In [7]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [8]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

## Save Trending Venues at Dinner time to CSV

In [ ]:
trending_venues_df.df.to_csv('TrendingDinnerLawrencePark.csv', index=False) 

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 


### Visualize trending venues


In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Lawrence Park',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [ ]:
# display map
venues_map

<a id="item6"></a>
